On extrait les jsons qui indique le nombre d'objet en mouvement et immobile d'après l'algorithme de détection et de suivi

In [1]:
from mouvement_label.preprocess import read_jsons,preprocess
from mouvement_label.detection_mouvement import detect_moving_frame,frame_by_frame
from mouvement_label.affichage import graph
from tqdm import tqdm

In [2]:
data_json_dict,frame_json_dict=read_jsons(PATH="./json/",num_cam="cam1",date="20240328")

un exemple

In [3]:
data_json_dict.keys(),frame_json_dict.keys(),data_json_dict.keys()==frame_json_dict.keys()

(dict_keys(['20240328135129', '20240328135329', '20240328105223', '20240328112839', '20240328113640', '20240328115042', '20240328115242', '20240328115442', '20240328121925', '20240328122125', '20240328122525', '20240328123525', '20240328123725', '20240328123925', '20240328125128', '20240328125328', '20240328131528', '20240328131928', '20240328132328', '20240328132528', '20240328133129', '20240328133329', '20240328133929']),
 dict_keys(['20240328135129', '20240328135329', '20240328105223', '20240328112839', '20240328113640', '20240328115042', '20240328115242', '20240328115442', '20240328121925', '20240328122125', '20240328122525', '20240328123525', '20240328123725', '20240328123925', '20240328125128', '20240328125328', '20240328131528', '20240328131928', '20240328132328', '20240328132528', '20240328133129', '20240328133329', '20240328133929']),
 True)

In [4]:
#moveless_object={}
#for date_mouvment_detecte in tqdm(data_json_dict.keys()):
#    data_json_dict[date_mouvment_detecte]=preprocess(data_json_dict[date_mouvment_detecte])
#    fbf,nb = frame_by_frame(frame_json_dict[date_mouvment_detecte],data_json_dict[date_mouvment_detecte])
#    moveless_object[date_mouvment_detecte]={"fbf":fbf,"nb":nb}

In [5]:
from mouvement_label.affichage import graph_nb
import os 
import json
#list_json=os.listdir("./result_json/result_json_cam1/")
#
#for json_file in list_json:
#    with open("./result_json/result_json_cam1/" + json_file, "r") as f:
#        data_json = json.load(f)
#    graph_nb(data_json,json_file[:-5])

In [6]:
#with open("./20240328.json","w") as f:
#    json.dump(moveless_object,f)

Annotation des vidéos

In [7]:
with open("./20240328.json",'r') as f:
    data_exp = json.load(f)

In [8]:
nb_moveless={}
for date_video in data_exp.keys():
    nb_moveless[date_video]=[]
    for frame in data_exp[date_video]["fbf"].keys():
        nb_moveless[date_video].append(len(data_exp[date_video]["fbf"][frame][1]))

In [9]:
import plotly.graph_objects as go 
import numpy as np 

window_size = 30

def median_filter_time_series(data, window_size):
    filtered_data = np.zeros_like(data)
    for i in range(len(data)):
        start_index = max(0, i - window_size // 2)
        end_index = min(len(data), i + window_size // 2 + 1)
        neighborhood = data[start_index:end_index]
        filtered_data[i] = np.median(neighborhood)
    return filtered_data

# Montre le filtrage médian / moyenne des signaux carrés
#for date_exp in nb_moveless.keys():
#    time = np.arange(1,len(nb_moveless[date_exp]))
#
#    motving_trace=list(data_exp[date_exp]["nb"].values())
#    smoothed_time_series = median_filter_time_series(nb_moveless[date_exp], 250)
#
#    smoothed_time_series_2=median_filter_time_series(motving_trace, 15)
#
#    movingless_trace_smoothed = go.Scatter(x=time, y=smoothed_time_series, mode='lines', name='motionless objects smoothed')
#    movingless_trace_smoothed_2 = go.Scatter(x=time, y=smoothed_time_series_2, mode='lines', name='motion objects smoothed')
#    movingless_trace = go.Scatter(x=time, y=nb_moveless[date_exp], mode='lines', name='motionless objects real')
#    motion_trace = go.Scatter(x=time, y=motving_trace, mode='lines', name='motion objects')
#    
#    # Create figure
#    fig = go.Figure()
#    
#    # Add traces to figure
#    fig.add_trace(movingless_trace)
#    fig.add_trace(movingless_trace_smoothed)
#    fig.add_trace(movingless_trace_smoothed_2)
#    fig.add_trace(motion_trace)
#    # Update layout
#    fig.update_layout(title=f'Moving vs Motionless Objects over Time - {date_exp}',
#                      xaxis_title='Time',
#                      yaxis_title='Number of Objects')
#
#    # Show figure
#    fig.show()

In [10]:
from collections import Counter
annotation={}
# Fonction pour détecter les segments de valeurs constantes
def segments_valeurs_constantes(serie):
    serie_t=np.array(serie.copy())[1:]
    serie_t_1 = np.array(serie.copy())[:-1]
    
    return serie_t-serie_t_1

for date in data_exp.keys():
    motion=np.array(list(data_exp[date]["nb"].values()))
    smoothed_time_series=median_filter_time_series(motion, 15)
    segments = segments_valeurs_constantes(smoothed_time_series)
    nb_passage = 0
    for num in segments:
        if num > 0:
            nb_passage += num 
    
    if np.any(motion):
        annotation[date]=f"{nb_passage} passage(s)"
    else : 
        annotation[date]=f"aucun passage"

In [11]:
annotation

{'20240328135129': 'aucun passage',
 '20240328135329': '2 passage(s)',
 '20240328105223': 'aucun passage',
 '20240328112839': '1 passage(s)',
 '20240328113640': 'aucun passage',
 '20240328115042': '3 passage(s)',
 '20240328115242': '1 passage(s)',
 '20240328115442': 'aucun passage',
 '20240328121925': '1 passage(s)',
 '20240328122125': '1 passage(s)',
 '20240328122525': '1 passage(s)',
 '20240328123525': '2 passage(s)',
 '20240328123725': '1 passage(s)',
 '20240328123925': '2 passage(s)',
 '20240328125128': '1 passage(s)',
 '20240328125328': '1 passage(s)',
 '20240328131528': '2 passage(s)',
 '20240328131928': '2 passage(s)',
 '20240328132328': '1 passage(s)',
 '20240328132528': 'aucun passage',
 '20240328133129': '1 passage(s)',
 '20240328133329': 'aucun passage',
 '20240328133929': '1 passage(s)'}

In [12]:
from mouvement_label.association_video_measurements import video_to_date,associate_measure,association
for date in annotation.keys():
    print(video_to_date(date))

2024-03-28 13:51:29+01:00
2024-03-28 13:53:29+01:00
2024-03-28 10:52:23+01:00
2024-03-28 11:28:39+01:00
2024-03-28 11:36:40+01:00
2024-03-28 11:50:42+01:00
2024-03-28 11:52:42+01:00
2024-03-28 11:54:42+01:00
2024-03-28 12:19:25+01:00
2024-03-28 12:21:25+01:00
2024-03-28 12:25:25+01:00
2024-03-28 12:35:25+01:00
2024-03-28 12:37:25+01:00
2024-03-28 12:39:25+01:00
2024-03-28 12:51:28+01:00
2024-03-28 12:53:28+01:00
2024-03-28 13:15:28+01:00
2024-03-28 13:19:28+01:00
2024-03-28 13:23:28+01:00
2024-03-28 13:25:28+01:00
2024-03-28 13:31:29+01:00
2024-03-28 13:33:29+01:00
2024-03-28 13:39:29+01:00


In [13]:
import pandas as pd

df=pd.read_csv("D:/IMT Nord Europe/2eme annee/P6/P6 - Système automatique d'annotation de données de pollution/P6_Systeme_annotation_donnees_polution/data/output_2024-03-29_10_24.csv")

In [14]:
#prend les events uniques détectés
events= set(list(annotation.values()))
#initialise l'ensemble des events à 0 pour toutes mesures
for event in events:
    df[event]=np.zeros(df.Date.shape).T
#print(df.columns)
for date in annotation.keys():
    print(date)
    mesure_indexes = associate_measure(date,5,df)
    print(mesure_indexes)
    for mesure_index in mesure_indexes:
      df[annotation[date]].iloc[mesure_index]=1
print(df.columns)

20240328135129
Index([ 309,  310,  311,  312,  313,  634,  635,  636,  637,  638,  869,  870,
        871,  872,  873, 1110, 1111, 1112, 1113, 1114],
      dtype='int64')
20240328135329
Index([ 311,  312,  313,  314,  315,  636,  637,  638,  639,  640,  871,  872,
        873,  874,  875, 1112, 1113, 1114, 1115, 1116],
      dtype='int64')
20240328105223
Index([ 46,  47, 130, 131, 132, 133, 134, 370, 371, 455, 456, 457, 458, 459,
       690, 691, 692, 693, 694, 931, 932, 933, 934, 935],
      dtype='int64')
20240328112839
Index([ 79,  80,  81,  82,  83, 166, 167, 168, 169, 170, 401, 402, 403, 491,
       492, 493, 494, 495, 726, 727, 728, 729, 730, 967, 968, 969, 970, 971],
      dtype='int64')
20240328113640
Index([174, 175, 176, 177, 178, 499, 500, 501, 502, 503, 734, 735, 736, 737,
       738, 975, 976, 977, 978, 979],
      dtype='int64')
20240328115042
Index([188, 189, 190, 191, 192, 513, 514, 515, 516, 517, 748, 749, 750, 751,
       752, 989, 990, 991, 992, 993],
      dtype='in

C:\Users\ElieS\AppData\Local\Temp\ipykernel_3080\1704920986.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[annotation[date]].iloc[mesure_index]=1
C:\Users\ElieS\AppData\Local\Temp\ipykernel_3080\1704920986.py:12: SettingWithCopyWarn

Index([ 233,  234,  235,  236,  237,  558,  559,  560,  561,  562,  793,  794,
        795,  796,  797, 1034, 1035, 1036, 1037, 1038],
      dtype='int64')
20240328123725
Index([ 235,  236,  237,  238,  239,  560,  561,  562,  563,  564,  795,  796,
        797,  798,  799, 1036, 1037, 1038, 1039, 1040],
      dtype='int64')
20240328123925
Index([ 237,  238,  239,  240,  241,  562,  563,  564,  565,  566,  797,  798,
        799,  800,  801, 1038, 1039, 1040, 1041, 1042],
      dtype='int64')
20240328125128
Index([ 249,  250,  251,  252,  253,  574,  575,  576,  577,  578,  809,  810,
        811,  812,  813, 1050, 1051, 1052, 1053, 1054],
      dtype='int64')
20240328125328
Index([ 251,  252,  253,  254,  255,  576,  577,  578,  579,  580,  811,  812,
        813,  814,  815, 1052, 1053, 1054, 1055, 1056],
      dtype='int64')
20240328131528
Index([ 273,  274,  275,  276,  277,  598,  599,  600,  601,  602,  833,  834,
        835,  836,  837, 1074, 1075, 1076, 1077, 1078],
      dtyp

C:\Users\ElieS\AppData\Local\Temp\ipykernel_3080\1704920986.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[annotation[date]].iloc[mesure_index]=1
C:\Users\ElieS\AppData\Local\Temp\ipykernel_3080\1704920986.py:12: SettingWithCopyWarn

In [29]:
for index in df.index:
    df["aucun passage"].iloc[index] = int(not(df["1 passage(s)"].iloc[index].item() or df["2 passage(s)"].iloc[index].item() or df["3 passage(s)"].iloc[index].item()))

C:\Users\ElieS\AppData\Local\Temp\ipykernel_3080\1799226572.py:2: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\ElieS\AppData\Local\Temp\ipykernel_3080\1799226572.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy

In [30]:
df

,Unnamed: 0,Date,parkName,sensorId,temperature,humidity,pressure,noise,noise_max,pm1,pm2_5,pm10,1 passage(s),aucun passage,2 passage(s),3 passage(s)
0,0,2024-03-28 09:56:33+01:00,FR-19997,866356067365675,22.0,39.0,9799.0,55.0,109.0,413.0,825.0,1318.0,0.0,1.0,0.0,0.0
1,1,2024-03-28 09:57:33+01:00,FR-19997,866356067365675,NaN,NaN,NaN,59.0,120.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0
2,2,2024-03-28 09:58:33+01:00,FR-19997,866356067365675,22.0,40.0,9799.0,54.0,116.0,413.0,825.0,1318.0,0.0,1.0,0.0,0.0
3,3,2024-03-28 09:59:33+01:00,FR-19997,866356067365675,NaN,NaN,NaN,53.0,87.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0
4,4,2024-03-28 10:00:33+01:00,FR-19997,866356067365675,22.0,40.0,9799.0,51.0,109.0,413.0,825.0,1318.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,235,2024-03-28 13:57:04+01:00,FR-19997,866833046798499,NaN,NaN,NaN,66.0,101.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0
1116,236,2024-03-28 13:58:04+01:00,FR-19997,866833046798499,17.0,40.0,9825.0,64.0,110.0,2062.0,3572.0,9168.0,0.0,0.0,1.0,0.0
1117,237,2024-03-28 13:59:04+01:00,FR-19997,866833046798499,NaN,NaN,NaN,66.0,106.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0
1118,238,2024-03-28 14:00:04+01:00,FR-19997,866833046798499,17.0,39.0,9824.0,66.0,104.0,2062.0,3572.0,9168.0,0.0,1.0,0.0,0.0


In [31]:
df_capteur_entreprise=df.dropna(axis=0)

In [32]:
df_capteur_entreprise

,Unnamed: 0,Date,parkName,sensorId,temperature,humidity,pressure,noise,noise_max,pm1,pm2_5,pm10,1 passage(s),aucun passage,2 passage(s),3 passage(s)
0,0,2024-03-28 09:56:33+01:00,FR-19997,866356067365675,22.0,39.0,9799.0,55.0,109.0,413.0,825.0,1318.0,0.0,1.0,0.0,0.0
2,2,2024-03-28 09:58:33+01:00,FR-19997,866356067365675,22.0,40.0,9799.0,54.0,116.0,413.0,825.0,1318.0,0.0,1.0,0.0,0.0
4,4,2024-03-28 10:00:33+01:00,FR-19997,866356067365675,22.0,40.0,9799.0,51.0,109.0,413.0,825.0,1318.0,0.0,1.0,0.0,0.0
6,6,2024-03-28 10:02:33+01:00,FR-19997,866356067365675,22.0,38.0,9798.0,52.0,90.0,445.0,651.0,898.0,0.0,1.0,0.0,0.0
8,8,2024-03-28 10:06:40+01:00,FR-19997,866356067365675,22.0,37.0,9800.0,52.0,101.0,514.0,514.0,1006.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,230,2024-03-28 13:52:04+01:00,FR-19997,866833046798499,16.0,40.0,9825.0,76.0,108.0,2144.0,3380.0,7934.0,0.0,1.0,0.0,0.0
1112,232,2024-03-28 13:54:04+01:00,FR-19997,866833046798499,16.0,41.0,9825.0,68.0,101.0,2154.0,2840.0,7230.0,0.0,0.0,1.0,0.0
1114,234,2024-03-28 13:56:04+01:00,FR-19997,866833046798499,16.0,40.0,9825.0,63.0,102.0,2154.0,2840.0,7230.0,0.0,0.0,1.0,0.0
1116,236,2024-03-28 13:58:04+01:00,FR-19997,866833046798499,17.0,40.0,9825.0,64.0,110.0,2062.0,3572.0,9168.0,0.0,0.0,1.0,0.0


In [33]:
def graphe(indicateur="pm2_5",df=df_capteur_entreprise):
    fig = go.Figure()
    for capteur in df["sensorId"].unique():
        capteur_name=str(capteur)
        if not (capteur_name.endswith("18") or capteur_name.endswith("75")): 
            fig.add_trace(go.Scatter(x=df[df["sensorId"]==capteur]['Date'], y=df[df["sensorId"]==capteur][indicateur], mode='lines',name=capteur_name))

    fig.update_layout(legend_title_text = f"{indicateur} vs time ")
    fig.update_xaxes(title_text="Time ")
    fig.update_yaxes(title_text=indicateur)
    fig.show()



In [34]:
graphe(df=df_capteur_entreprise)

In [36]:
graphe(indicateur="1 passage(s)",df=df_capteur_entreprise)

In [37]:
graphe(indicateur="2 passage(s)",df=df_capteur_entreprise)

In [38]:
graphe(indicateur="3 passage(s)",df=df_capteur_entreprise)